**Import librarys**

In [1]:
from ultralytics import YOLO
from ultralytics import settings
import torch
import os
import cv2

**Check torch version**

In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))
print("Antal CPU-kärnor:", os.cpu_count())

**Check Ultralytics settings**

In [ ]:
print(settings)

**Download Dataset**

In [5]:
from roboflow import Roboflow

# dotenv.load_dotenv()
# settings.reset()

# ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")
# PROJECT_ID = os.getenv("PROJECT_OBJECT_DETECTION")
# VERSION = os.getenv("OBJECT_DETECTION_MODEL_VERSION")

# def download_dataset(PROJECT_ID, version, yolov):
#   rf = Roboflow(api_key=ROBOFLOW_API_KEY)
#   project = rf.workspace().project(PROJECT_ID)
# #   version = project.version(version)
#   dataset = project.version(version).download(yolov)
#   return dataset

# dataset = download_dataset(PROJECT_ID, 9, yolov="yolov11")



https://docs.ultralytics.com/modes/train/#train-settings

**Train model**

In [ ]:
def model_training(data):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO("yolo11n.pt")  # Laddar in förtränad modell
    model.to(device)
    results = model.train(
        data=data, 
        epochs=100, 
        imgsz=640, 
        batch=8,
        workers=os.cpu_count(),
        exist_ok=True
        )
    return results

results = model_training(data="modelv12.yaml")


**Validate model**

In [ ]:
def model_validation():
    model = YOLO("runs/detect/train/weights/best.pt")
    metrics = model.val(
        data="modelv12.yaml", 
        imgsz=640, 
        batch=8, 
        conf=0.25, 
        iou=0.6, 
        device="0",
        dnn=True, 
        save_json=True)
    return metrics

modelv12_path = os.path.join(os.getcwd(), "modelv12")
metrics = model_validation()
print(metrics.box.map)
print(metrics.box.map50)
print(metrics.box.map75)
print(metrics.box.maps)


**Test model**

**Prediction**

In [ ]:
def predict_image(image_path, output_path="output.png", conf=0.5):

    model = YOLO("runs/detect/train/weights/best.pt")
    
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    results = model.predict(
        img, 
        conf=conf, 
        stream=True
    )

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = [int(coord.item()) for coord in box.xyxy[0]]
            label = int(box.cls.item())
            confidence = box.conf.item()
            
            # Rita bounding boxen på bilden
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"Label: {label}, Conf: {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imwrite(output_path, img)
    print(f"Bilden har sparats till {output_path}")


test_image = os.path.join(os.getcwd(), "test", "board_003.png")

predict_image(test_image, "output_with_boxes.png")
